# **INTRODUCTIONS**

---

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_DECEASED_FORCAST_PER_SAFETY_REGION]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTAGE_CBS_DECEASED_FORCAST_PER_SAFETY_REGION]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION]'))
CREATE TABLE [VWSSTAGE].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION](
    [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_DECEASED_FORCAST_PER_SAFETY_REGION]),
  	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [WEEK] [VARCHAR](100) NULL,
    [SEASONAL_PATTERN_WITH_MARGINS] [VARCHAR](100) NULL,
    [UPPER_LIMIT] [VARCHAR](100) NULL,
    [LOWER_LIMIT] [VARCHAR](100) NULL,
    [VR_NAME] [VARCHAR](100) NULL,
    [VR_CODE] [VARCHAR](100) NULL
);
GO

-- 3) CREATE STAGE INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_DECEASED_FORECAST_PER_SAFETY_REGION')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_DECEASED_FORECAST_PER_SAFETY_REGION] ON [VWSSTAGE].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION] (
  [DATE_LAST_INSERTED]
) INCLUDE (
  [WEEK], 
  [SEASONAL_PATTERN_WITH_MARGINS], 
  [LOWER_LIMIT], 
  [UPPER_LIMIT], 
  [VR_NAME], 
  [VR_CODE]
);
GO

### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE INTER SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSINTER_CBS_DECEASED_FORECAST_PER_SAFETY_REGION]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSINTER_CBS_DECEASED_FORECAST_PER_SAFETY_REGION]
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE INTER TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION]'))
CREATE TABLE [VWSINTER].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION](
    [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSINTER_CBS_DECEASED_FORECAST_PER_SAFETY_REGION]),
	  [DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE(),
    [YEAR] [INT] NULL,
    [WEEK] [INT] NULL,
    [DECEASED_FORECAST] [INT] NULL,
    [DECEASED_FORECAST_LOW] [INT] NULL,
    [DECEASED_FORECAST_HIGH] [INT] NULL,
    [VR_NAME] [VARCHAR](100),
    [VR_CODE] [VARCHAR](100)
);
GO

-- 3) CREATE INTER INDEX(ES).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCIX_DLI_CBS_DECEASED_FORECAST_PER_SAFETY_REGION')
CREATE NONCLUSTERED INDEX [NCIX_DLI_CBS_DECEASED_FORECAST_PER_SAFETY_REGION] ON [VWSINTER].[CBS_DECEASED_VR_FORECAST](
  [DATE_LAST_INSERTED]
)
INCLUDE (
  [YEAR], 
  [WEEK], 
  [DECEASED_FORECAST], 
  [DECEASED_FORECAST_LOW], 
  [DECEASED_FORECAST_HIGH], 
  [VR_NAME], 
  [VR_CODE]
);
GO


# **STORE PROCEDURES**
---

### STAGE &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT. 
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_DECEASED_FORECAST_PER_SAFETY_REGION_STAGE_TO_INTER]
AS
BEGIN
    INSERT INTO [VWSINTER].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION] (
        [YEAR],
        [WEEK],
        [DECEASED_FORECAST],
        [DECEASED_FORECAST_LOW],
        [DECEASED_FORECAST_HIGH],
        [VR_NAME],
        [VR_CODE] 
    )
    SELECT
        CAST(SUBSTRING([WEEK], 1,4) AS INT) AS [YEAR],
        CAST(SUBSTRING([WEEK], 11,2) AS INT) AS [WEEK],
        [SEASONAL_PATTERN_WITH_MARGINS],
        [LOWER_LIMIT],
        [UPPER_LIMIT],
        [VR_NAME],
        [VR_CODE] 
    FROM 
        [VWSSTAGE].[CBS_DECEASED_FORECAST_PER_SAFETY_REGION]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_DECEASED_VR_FORECAST])
END;
GO

# **DROP ARTIFACTS**

---

### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

DECLARE @workflow_name NVARCHAR(50) = 'CBS_DECEASED_VR_FORECAST';

-- 1) DROP DEPENDENC(Y)(IES).....    
DELETE FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES]
WHERE [ID] IN (SELECT dependencies.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
    WHERE dependencies.[NAME] IN (
        'DEPENDENCY_PROCESS_SP_INTER_CBS_DECEASED_FORECAST_VR',
        'DEPENDENCY_PROCESS_SP_DEST_CBS_DECEASED_FORECAST_VR'
    ) AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
);

-- 3) DROP PROCESS(ES)......
WITH CTE 
AS
(
    SELECT
        v_processes.[PROCESS_NAME],
        processes.[ID]
    FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
    INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
    WHERE v_processes.[PROCESS_NAME] IN (
        'PROCESS_SP_DEST_CBS_DECEASED_FORECAST_VR',
        'PROCESS_SP_INTER_CBS_DECEASED_FORECAST_VR',
        'PROCESS_LOAD_CBS_DECEASED_FORECAST_STAGE_VR',
        @workflow_name
    ) AND v_workflows.[NAME] = @workflow_name
)
DELETE [DATATINO_ORCHESTRATOR_1].[PROCESSES] 
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
WHERE dataflows.[NAME] IN (SELECT [PROCESS_NAME] FROM CTE) 
    AND processes.[ID] IN (SELECT [ID] FROM CTE);

-- 4) DROP SOURCE(S).....
DELETE FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] IN (
    'SOURCE_SP_DEST_CBS_DECEASED_VR_DEST',
    'SOURCE_SP_INTER_CBS_DECEASED_VR_FORECAST_INTER',
    CONCAT('SOURCE_LOAD_', @workflow_name, '_STAGE')
);

-- 5) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );
GO

-- 6) DROP WORKFLOW(S).....
DELETE [DATATINO_ORCHESTRATOR_1].[WORKFLOWS]
FROM [DATATINO_ORCHESTRATOR_1].[DATAFLOWS] dataflows, [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
WHERE dataflows.[NAME] = @workflow_name
    AND workflows.[ID] IN (
        SELECT
            workflows.[ID]
        FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
        INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] 
            AND v_workflows.[ID] = workflows.[ID]
        WHERE v_workflows.[NAME] = @workflow_name
    );
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CBS_DECEASED_FORECAST_PER_SAFETY_REGION',
        @workflow NVARCHAR(50) = 'CBS_DECEASED_PER_SAFETY_REGION',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow,
    @description = @workflow_description,
    @SCHEDULE = '0 1 * * FRI', -- AT 01:00 AM, ONLY ON FRIDAY
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....

-- PREVIOUSLY @source = datafiles/CBS/verwachte_sterfte_2020_en_2021.csv

SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/CBS/verwachte_sterfte_2020_2021_en_2022.csv'
    WHEN 'acceptance' THEN 'datafiles/CBS/verwachte_sterfte_2020_2021_en_2022.csv'
    ELSE 'datafiles/CBS/verwachte_sterfte_2020_2021_en_2022.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Week|Seizoenspatroon met marges|Ondergrens|Bovengrens|VR naam|VR Code',
    @target_columns = 'WEEK|SEASONAL_PATTERN_WITH_MARGINS|UPPER_LIMIT|LOWER_LIMIT|VR_NAME|VR_CODE|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = CONCAT('dbo.SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;
    
-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST......
SET @source = CONCAT('dbo.SP_', @workflow_name, '_INTER_TO_DEST');
SET @source_name = CONCAT('SOURCE_SP_', @workflow_name, '_INTER_TO_DEST');
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST'); 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name);
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
SET @process_source_name = CONCAT('SOURCE_SP_', @workflow_name, '_STAGE_TO_INTER');

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
SET @dependency_dataflow_name = CONCAT('PROCESS_SP_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name);
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO